In [1]:
from bs4 import BeautifulSoup
import requests


In [1]:
import os
os.system("pip install git+https://github.com/openai/whisper.git")
import gradio as gr
from bs4 import BeautifulSoup
import requests
from newspaper import Article
from newspaper import Config
import gradio as gr
from gradio.mix import Parallel, Series
import nltk
 
#from convert import main

from transformers import AutoTokenizer
from transformers import pipeline
  
 
nltk.download('punkt')

def extract_article_text(url):
  USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'
  config = Config()
  config.browser_user_agent = USER_AGENT
  config.request_timeout = 10

  article = Article(url, config=config)
  article.download()
  article.parse()
  text = article.text
  return text

extractor = gr.Interface(extract_article_text, 'text', 'text')
summarizer = gr.Interface.load("huggingface/facebook/bart-large-cnn")

sample_url = [['https://www.cnbc.com/2022/11/02/fed-hikes-by-another-three-quarters-of-a-point-taking-rates-to-the-highest-level-since-january-2008.html'],
               
    ]
           
description =  '''
        Note: A news article behind a paywall may produce an error.
        '''
  
iface = Series(extractor, summarizer, 
  inputs = gr.inputs.Textbox(
      lines = 2,
      label = 'URL'
  ),
  outputs = 'text',
  title = 'News Summarizer',
  theme = 'peach',
  description = description,
  examples=sample_url)
  
iface.launch(share=True)

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-hjswwftf


  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-hjswwftf


  Resolved https://github.com/openai/whisper.git to commit 9f70a352f9f8630ab3aa0d06af5cb9532bd8c21d
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


2022-11-03 18:32:50.046536: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-03 18:32:50.181435: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-03 18:32:50.217680: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-03 18:32:50.847866: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

Fetching model from: https://huggingface.co/facebook/bart-large-cnn


/home/ec2-user/anaconda3/envs/deep-learning/lib/python3.9/site-packages/gradio/inputs.py:26: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/home/ec2-user/anaconda3/envs/deep-learning/lib/python3.9/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/ec2-user/anaconda3/envs/deep-learning/lib/python3.9/site-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/ec2-user/anaconda3/envs/deep-learning/lib/python3.9/site-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)
/home/ec2-user/anaconda3/envs/deep-learning/lib/python3.9/site-packages/gradio/interface.py:328: UserWarning: Currently, only the 'default' theme is suppor

Running on local URL:  http://127.0.0.1:7863
Running on public URL: https://62ce32cf78a3406a.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7fdc8530e940>,
 'http://127.0.0.1:7863/',
 'https://62ce32cf78a3406a.gradio.app')

In [ ]:

URL = "https://www.vogue.com/article/latinx-beauty-brands"
#URL = "https://hackernoon.com/will-the-game-stop-with-gamestop-or-is-this-just-the-beginning-2j1x32aa"

In [ ]:


r = requests.get(URL)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [ ]:
ARTICLE

In [ ]:
max_chunk = 500

In [ ]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [ ]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

In [ ]:

len(chunks)

In [ ]:

res = summarizer(chunks, max_length=200, min_length=30, do_sample=False)

In [ ]:
' '.join([summ['summary_text'] for summ in res])

In [ ]:
text = ' '.join([summ['summary_text'] for summ in res])

In [ ]:

with open('blogsummary.txt', 'w') as f:
    f.write(text)